# Hello, FABRIC: Create your first FABRIC slice


## Setup the Experiment

#### Import the FABRIC API

In [2]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()
                     
fablib.show_config()

Log Level,INFO
Log File,/tmp/fablib/fablib.log
Data directory,/tmp/fablib
SSH Command Line,ssh -i {{ _self_.private_ssh_key_file }} -F /home/vscode/work/fabric_config/ssh_config {{ _self_.username }}@{{ _self_.management_ip }}
Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/vscode/.tokens.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net


Log Level,INFO
Log File,/tmp/fablib/fablib.log
Data directory,/tmp/fablib
SSH Command Line,ssh -i {{ _self_.private_ssh_key_file }} -F /home/vscode/work/fabric_config/ssh_config {{ _self_.username }}@{{ _self_.management_ip }}
Orchestrator,orchestrator.fabric-testbed.net
Credential Manager,cm.fabric-testbed.net
Core API,uis.fabric-testbed.net
Artifact Manager,artifacts.fabric-testbed.net
Token File,/home/vscode/.tokens.json
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
Bastion Host,bastion.fabric-testbed.net


#### (Optional) Query Available Resources

This optional command queries the FABRIC services to find the available resources. It may be useful for finding a site with available capacity.

In [3]:
try:
    available_resources = fablib.get_available_resources()
    print(f"Available Resources: {available_resources}")
except Exception as e:
    print(f"Error: {e}")

KeyboardInterrupt: 

## Create the Experiment Slice

In [50]:
# Configuration variables
slice_name = 'K8s_on_FABRIC'
site = "UCSD"
image = 'default_ubuntu_20'
network_name = 'NET1'

# Create lists for node configurations
nic_names, node_names = [], []
for i in range(1, 4):
    if i == 1:
        node_names.append(f"cpnode")
        nic_names.append(f"NIC1")
    else:
        node_names.append(f"wknode{i-1}")
        nic_names.append(f"NIC{i}")

print(f"Site: {site}")

try:
    # Create Slice
    slice = fablib.new_slice(slice_name)
    
    # Add nodes and interfaces
    nodes = []
    ifaces = []
    for i in range(len(node_names)):
        node = slice.add_node(
            name=node_names[i],
            site=site,
            image=image,
            disk=100
        )
        nodes.append(node)
        iface = node.add_component(
            model='NIC_Basic', 
            name=nic_names[i]
        ).get_interfaces()[0]
        ifaces.append(iface)
    
    # Network
    net1 = slice.add_l3network(
        name=network_name, 
        interfaces=ifaces, 
        type='IPv4'
    )
    
    # Submit Slice Request
    slice_id = slice.submit()

except Exception as e:
    print(f"{e}")


Retry: 10, Time: 259 sec


ID,f45ef733-64be-4c1d-9398-0fa7fced6a3a
Name,K8s_on_FABRIC
Lease Expiration (UTC),2025-03-14 16:38:21 +0000
Lease Start (UTC),2025-03-13 16:38:21 +0000
Project ID,bbe0d94c-736b-477a-a2e6-fef9fe7ac9ca
State,StableOK


ID,Name,Cores,RAM,Disk,Image,Image Type,Host,Site,Username,Management IP,State,Error,SSH Command,Public SSH Key File,Private SSH Key File
6132751a-83b4-4daf-be14-7ba0ded7737c,cpnode,2,8,100,default_ubuntu_20,qcow2,ucsd-w1.fabric-testbed.net,UCSD,ubuntu,132.249.252.151,Active,,ssh -i /home/vscode/.ssh/fabric_local/silver -F /home/vscode/work/fabric_config/ssh_config ubuntu@132.249.252.151,/home/vscode/.ssh/fabric_local/silver.pub,/home/vscode/.ssh/fabric_local/silver
9efad2f0-6399-488b-b005-888e2ec5aa62,wknode1,2,8,100,default_ubuntu_20,qcow2,ucsd-w1.fabric-testbed.net,UCSD,ubuntu,132.249.252.165,Active,,ssh -i /home/vscode/.ssh/fabric_local/silver -F /home/vscode/work/fabric_config/ssh_config ubuntu@132.249.252.165,/home/vscode/.ssh/fabric_local/silver.pub,/home/vscode/.ssh/fabric_local/silver
df11f0e8-f471-4fc1-b15e-9b8aa9004be3,wknode2,2,8,100,default_ubuntu_20,qcow2,ucsd-w1.fabric-testbed.net,UCSD,ubuntu,132.249.252.182,Active,,ssh -i /home/vscode/.ssh/fabric_local/silver -F /home/vscode/work/fabric_config/ssh_config ubuntu@132.249.252.182,/home/vscode/.ssh/fabric_local/silver.pub,/home/vscode/.ssh/fabric_local/silver


ID,Name,Layer,Type,Site,Subnet,Gateway,State,Error
5363e492-babe-420c-bf9b-0ab51a0ebe99,NET1,L3,FABNetv4,UCSD,10.134.132.0/24,10.134.132.1,Active,


Name,Short Name,Node,Network,Bandwidth,Mode,VLAN,MAC,Physical Device,Device,IP Address,Numa Node,Switch Port
cpnode-NIC1-p1,p1,cpnode,NET1,100,config,,02:34:D8:94:66:FD,enp7s0,enp7s0,fe80::34:d8ff:fe94:66fd,6,HundredGigE0/0/0/5
wknode1-NIC2-p1,p1,wknode1,NET1,100,config,,02:74:7A:C2:0A:63,enp7s0,enp7s0,fe80::74:7aff:fec2:a63,6,HundredGigE0/0/0/5
wknode2-NIC3-p1,p1,wknode2,NET1,100,config,,0A:19:42:43:96:03,enp7s0,enp7s0,fe80::819:42ff:fe43:9603,6,HundredGigE0/0/0/5



Time to print interfaces 270 seconds


### Print the Node's Attributes

Each node in the slice has a set of get functions that return the node's attributes.

In [51]:
try:
    slice = fablib.get_slice(slice_id=slice_id)
    nodes = slice.get_nodes()
    node_dict = {}
    address_dict = {}
    
    # Create node and address variables
    for i, node in enumerate(nodes, 1):
        node_name = node_names[i-1]
        node_dict[node_name] = node
        globals()[node_name] = node
        address_dict[f"{node_name}_address"] = None
        globals()[f"{node_name}_address"] = None
        print(f"{node_name}: {node}")

    # Get network info
    network = slice.get_network(name=network_name)
    network_available_ips = network.get_available_ips()
    print(f"Network: {network}")
    
    # Configure IPs for all nodes
    for node_name in node_names:
        node = node_dict[node_name]
        node_iface = node.get_interface(network_name=network_name)
        
        # Assign and store IP address
        node_address = network_available_ips.pop(0)
        node_iface.ip_addr_add(addr=node_address, subnet=network.get_subnet())
        address_dict[f"{node_name}_address"] = node_address
        globals()[f"{node_name}_address"] = node_address
        
        # Print node config
        print(f"\nConfiguration for {node_name}:")
        print(f"Address assigned: {node_address}")
        stdout, _ = node.execute(f'ip addr show {node_iface.get_os_interface()}')
        print(stdout)
        stdout, _ = node.execute('ip route list')
        print(stdout)

    # Print summary
    print("\nNode and Address Summary:")
    for node_name in node_names:
        print(f"{node_name}: {globals()[node_name]}")
        print(f"{node_name}_address: {globals()[f'{node_name}_address']}")

except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

cpnode: -----------------  -----------------------------------------------------------------------------------------------------------------
ID                 6132751a-83b4-4daf-be14-7ba0ded7737c
Name               cpnode
Cores              2
RAM                8
Disk               100
Image              default_ubuntu_20
Image Type         qcow2
Host               ucsd-w1.fabric-testbed.net
Site               UCSD
Management IP      132.249.252.151
Reservation State  Active
Error Message
SSH Command        ssh -i /home/vscode/.ssh/fabric_local/silver -F /home/vscode/work/fabric_config/ssh_config ubuntu@132.249.252.151
-----------------  -----------------------------------------------------------------------------------------------------------------
wknode1: -----------------  -----------------------------------------------------------------------------------------------------------------
ID                 9efad2f0-6399-488b-b005-888e2ec5aa62
Name               wknode1
Cores         


## Start the control plane
We follow the instructions that we have here: https://github.com/apache/openwhisk-deploy-kube/blob/master/docs/k8s-diy-ubuntu.md

In [52]:
try:
    file_attributes = cpnode.upload_file(local_file_path="/workspaces/local-fabric/FabricPortal_tests/config_control_plane.sh", remote_file_path="config_control_plane.sh")
    
    stdout, stderr = cpnode.execute(f"chmod +x config_control_plane.sh && ./config_control_plane.sh {network.get_subnet()} {cpnode_address}")

except Exception as e:
    print(f"Exception: {e}")
 



Get:1 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:3 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3431 kB]
Get:6 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 Packages [8628 kB]
Get:7 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe Translation-en [5124 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [500 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3481 kB]
Get:10 http://nova.clouds.archive.ubuntu.com/ubuntu focal/universe amd64 c-n-f Metadata [265 kB]
Get:11 http://nova.clouds.archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [144 kB]
Get:12 http://nova.clouds.archive.ubuntu.com/u

## Start the worker node
Put the join command in the `join_cmd` variable. The command should look like this: ` kubeadm join <control-plane-ip>:6443 --token <token> --discovery-token-ca-cert-hash sha256:<hash>`

In [53]:
join_cmd = "sudo kubeadm join 10.134.132.2:6443 --token 90w1cf.doxb7r3u1gff3gld \
	--discovery-token-ca-cert-hash sha256:ac9278bfb9d6a12fc03f70e375d172035d6707afe0a8053f9731c9f362be5079"
try:
    # Loop through worker nodes in node_dict
    for node_name, node in node_dict.items():
        if node_name == 'cpnode':  # Skip control plane node
            continue
            
        print(f"\nConfiguring {node_name}...")
        
        # Upload and execute config script
        try:
            print(f"Uploading and executing config_worker_node.sh on {node_name}...")
            file_attributes = node.upload_file(
                local_file_path="/workspaces/local-fabric/FabricPortal_tests/config_worker_node.sh", 
                remote_file_path="config_worker_node.sh"
            )
            exec_cmd = f"chmod +x config_worker_node.sh && ./config_worker_node.sh {join_cmd}"
            stdout, stderr = node.execute(exec_cmd)
            print(f"Config output for {node_name}:", stdout)
        except Exception as e:
            print(f"Failed to configure {node_name}: {e}")
            continue  # Skip to next node if configuration fails

except Exception as e:
    print(f"Main exception: {e}")
    traceback.print_exc()


Configuring wknode1...
Uploading and executing config_worker_node.sh on wknode1...


Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [128 kB]
Get:2 http://nova.clouds.archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:3 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3431 kB]
Get:4 http://nova.clouds.archive.ubuntu.com/ubuntu focal-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu focal-security/main Translation-en [500 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [3481 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/restricted Translation-en [487 kB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1034 kB]
Get:9 http://nova.clouds.archive.ubuntu.com/ubuntu focal-backports InRelease [128 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/universe Translation-en [219 kB]
Get:11 http://security.ubuntu.com/ubuntu focal-security/universe a

## Monitor the kubernetes cluster

In [41]:
try:
    # Get nodes
    print("Getting nodes...")
    stdout, stderr = cpnode.execute("kubectl get nodes")
    print(f"Nodes:\n{stdout}")
    if stderr:
        print(f"Stderr: {stderr}")

    # Get all resources across namespaces
    print("Getting all kubernetes resources...")
    stdout, stderr = cpnode.execute("kubectl get all --all-namespaces")
    print(f"All resources:\n{stdout}")
    if stderr:
        print(f"Stderr: {stderr}")
    
    # Get all pods with more details
    print("\nGetting detailed pods status across all namespaces...")
    stdout, stderr = cpnode.execute("kubectl get pods -A -o wide")
    print(f"Detailed pods status:\n{stdout}")
    if stderr:
        print(f"Stderr: {stderr}")
        
    # Describe kube-system namespace pods
    print("\nDescribing kube-system pods...")
    stdout, stderr = cpnode.execute("kubectl describe pods -n kube-system")
    print(f"Kube-system pods details:\n{stdout}")
    if stderr:
        print(f"Stderr: {stderr}")
    
    # Get logs from specific system pods
    system_components = ['kube-apiserver', 'kube-controller-manager', 'kube-scheduler', 'etcd']
    
    for component in system_components:
        print(f"\nGetting logs for {component}...")
        try:
            # First get the pod name
            cmd = f"kubectl get pods -n kube-system -l component={component} -o jsonpath='{{.items[0].metadata.name}}'"
            stdout, stderr = cpnode.execute(cmd)
            if stdout:
                pod_name = stdout.strip()
                # Then get the logs
                stdout, stderr = cpnode.execute(f"kubectl logs -n kube-system {pod_name} --tail=50")
                print(f"Logs from {component} ({pod_name}):\n{stdout}")
            else:
                print(f"No pod found for component {component}")
        except Exception as e:
            print(f"Error getting logs for {component}: {e}")

except Exception as e:
    print(f"Exception while monitoring kubernetes cluster: {e}")
    traceback.print_exc()



Getting nodes...
NAME      STATUS     ROLES           AGE     VERSION
cpnode    Ready      control-plane   2m41s   v1.31.7
wknode1   Ready      <none>          72s     v1.31.7
wknode2   NotReady   <none>          20s     v1.31.7
Nodes:
NAME      STATUS     ROLES           AGE     VERSION
cpnode    Ready      control-plane   2m41s   v1.31.7
wknode1   Ready      <none>          72s     v1.31.7
wknode2   NotReady   <none>          20s     v1.31.7

Getting all kubernetes resources...
NAMESPACE     NAME                                           READY   STATUS     RESTARTS   AGE
kube-system   pod/calico-kube-controllers-6879d4fcdc-tzcth   1/1     Running    0          2m35s
kube-system   pod/calico-node-bf4b2                          0/1     Init:1/3   0          21s
kube-system   pod/calico-node-d8djr                          1/1     Running    0          2m35s
kube-system   pod/calico-node-rxxmr                          1/1     Running    0          74s
kube-system   pod/coredns-7c65d6cfc9

## Node has successfully joined the cluster.

# Deploying a hello world application.

## First, we pull a hello world image and create a "deployment".

In [28]:
try:
    stdout, stderr = cpnode.execute("kubectl create deployment kubernetes-bootcamp --image=gcr.io/google-samples/kubernetes-bootcamp:v1 --replicas=9")
    print(f"stdout: {stdout}")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
 

deployment.apps/kubernetes-bootcamp created
stdout: deployment.apps/kubernetes-bootcamp created

stderr: 


## Now let's do some status commands.

In [134]:
try:
    _, stderr = cpnode.execute("kubectl get pods")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

NAMESPACE     NAME                                       READY   STATUS    RESTARTS   AGE
default       kubernetes-bootcamp-68cfbdbb99-w2pk4       1/1     Running   0          14s
kube-system   calico-kube-controllers-6879d4fcdc-rrp2b   1/1     Running   0          6m40s
kube-system   calico-node-kjxnc                          1/1     Running   0          3m44s
kube-system   calico-node-xrzxv                          1/1     Running   0          6m40s
kube-system   coredns-7c65d6cfc9-lsqw4                   1/1     Running   0          6m40s
kube-system   coredns-7c65d6cfc9-tqzzx                   1/1     Running   0          6m40s
kube-system   etcd-node1                                 1/1     Running   0          6m47s
kube-system   kube-apiserver-node1                       1/1     Running   0          6m47s
kube-system   kube-controller-manager-node1              1/1     Running   0          6m47s
kube-system   kube-proxy-472xw                           1/1     Running   0        

## Pick the correct pod name and run the command below.

In [29]:

try:
    _, stderr = cpnode.execute("kubectl describe pod kubernetes-bootcamp-68cfbdbb99-d67sb")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

Name:             kubernetes-bootcamp-68cfbdbb99-d67sb
Namespace:        default
Priority:         0
Service Account:  default
Node:             node3/10.20.5.91
Start Time:       Tue, 12 Nov 2024 03:47:12 +0000
Labels:           app=kubernetes-bootcamp
                  pod-template-hash=68cfbdbb99
Annotations:      cni.projectcalico.org/containerID: 06ee87410189c92c674d921c8a11167abab926f81d6f74da7c115f909272d113
                  cni.projectcalico.org/podIP: 10.146.2.113/32
                  cni.projectcalico.org/podIPs: 10.146.2.113/32
Status:           Running
IP:               10.146.2.113
IPs:
  IP:           10.146.2.113
Controlled By:  ReplicaSet/kubernetes-bootcamp-68cfbdbb99
Containers:
  kubernetes-bootcamp:
    Container ID:   containerd://5c233800d4a748ec92341a274e4d2e158872d10d18e214ac97d67c12a27119f3
    Image:          gcr.io/google-samples/kubernetes-bootcamp:v1
    Image ID:       gcr.io/google-samples/kubernetes-bootcamp@sha256:0d6b8ee63bb57c5f5b6156f446b3bc3b3c143d

## The next thing we need to do is to create what is called a "service".

We are going to use it to expose the deployment to the outside, through a port, which is 8080. Like this:

Note that the service itself will still need to be exposed. There's another "expose" step that we need to make.

In [30]:
try:
    _, stderr = cpnode.execute('kubectl expose deployment/kubernetes-bootcamp --type="ClusterIP" --port 8080')
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

service/kubernetes-bootcamp exposed
stdout: service/kubernetes-bootcamp exposed

stderr: 


## Let's check if the service was created.

In [31]:
try:
    _, stderr = cpnode.execute("kubectl get service kubernetes-bootcamp")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

NAME                  TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
kubernetes-bootcamp   ClusterIP   10.111.180.18   <none>        8080/TCP   11s
stdout: NAME                  TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)    AGE
kubernetes-bootcamp   ClusterIP   10.111.180.18   <none>        8080/TCP   11s

stderr: 


## Finally, we need to run a port forwarding command in order to expose the service to the outside.

Modify the --address flag. Use the "CLUSTER-IP" that is output form the command above.

In [32]:
try:
    _, stderr = cpnode.execute("kubectl port-forward --address 10.111.180.18 service/kubernetes-bootcamp 8080:8080 > /dev/null 2>&1 &")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

stdout: 
stderr: 


## Now our application should finally be visible. Let's test the deployment on the control plane machine itself.

In [35]:
try:
    _, stderr = cpnode.execute("curl 10.111.180.18:8080")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-68cfbdbb99-bdrtr | v=1
   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    84    0    84    0     0  14000      0 --:--:-- --:--:-- --:--:-- 14000
 stdout: Hello Kubernetes bootcamp! | Running on: kubernetes-bootcamp-68cfbdbb99-bdrtr | v=1

stderr:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    84    0    84    0     0  14000      0 --:--:-- --:--:-- --:--:-- 14000



# E2SAR Perf application in kubernetes test







## Run the perf test

In [80]:
# Run the perf test
# upload the yaml file to the control plane
file_attributes = cpnode.upload_file(local_file_path="/workspaces/local-fabric/FabricPortal_tests/e2sar-perf-test/e2sar-headless-fabric.yaml", remote_file_path="e2sar-headless-fabric.yaml")

# Apply the yaml file
try:
    _, stderr = cpnode.execute("kubectl apply -f e2sar-headless-fabric.yaml")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")

namespace/e2sar-perf created
persistentvolume/logs-pv created
persistentvolumeclaim/logs-pvc created
service/e2sar-receiver-svc created
networkpolicy.networking.k8s.io/e2sar-network-policy created
job.batch/e2sar-receiver created
job.batch/e2sar-sender created
job.batch/e2sar-log-viewer created
stderr: 


## Monitor the perf test

In [88]:
# To monitor sender logs directly on the sender node
# Define sender and receiver nodes from the slice
try:
    # Assuming we have nodes with specific roles or labels
    # You might need to adjust this based on your actual node naming/labeling
    sender_node = slice.get_node(name="wknode1")  # or any specific node name for sender
    receiver_node = slice.get_node(name="wknode2")  # or any specific node name for receiver
    
    # If nodes don't have specific names, you could use indices
    # sender_node = slice.get_nodes()[1]  # First worker node
    # receiver_node = slice.get_nodes()[2]  # Second worker node
    
    print(f"Sender node: {sender_node.get_name()}")
    print(f"Receiver node: {receiver_node.get_name()}")
except Exception as e:
    print(f"Error defining sender/receiver nodes: {e}")

print("--- sender"*10)

try:
    _, stderr = sender_node.execute("cat /tmp/e2sar-logs/sender/sender.log")
    print(f"Sender logs:")
    print(stderr)
except Exception as e:
    print(f"Exception when accessing sender logs: {e}")

print("--- receiver"*10)

# To monitor receiver logs directly on the receiver node
try:
    _, stderr = receiver_node.execute("cat /tmp/e2sar-logs/receiver/receiver.log")
    print(f"Receiver logs:")
    print(stderr)
except Exception as e:
    print(f"Exception when accessing receiver logs: {e}")


Sender node: wknode1
Receiver node: wknode2
--- sender--- sender--- sender--- sender--- sender--- sender--- sender--- sender--- sender--- sender
E2SAR Version: 0.1.4
Control plane                OFF
Event rate reporting in Sync ON
Using usecs as event numbers ON
*** Make sure the URI reflects proper data address, other parts are ignored.
Sending bit rate is 0.1 Gbps
Event size is 512 bytes or 4096 bits
Event rate is 24414.1 Hz
Inter-event sleep time is 40 microseconds
Sending 500 event buffers
Using MTU 512
Completed, 978 frames sent, 0 errors
Stopping threads
Sender logs:

--- receiver--- receiver--- receiver--- receiver--- receiver--- receiver--- receiver--- receiver--- receiver--- receiver
E2SAR Version: 0.1.4
Control plane will be OFF
Using Unassigned Threads
Will run for 300 sec
*** Make sure the URI reflects proper data address, other parts are ignored.
Receiving on ports 19522:19522
Stats:
	Events Received: 0
	Events Mangled: 0
	Events Lost: 0
	Data Errors: 0
	gRPC Errors: 0
	Ev

## Clean up the perf test

In [79]:
# Clean up the perf test
try:
    _, stderr = cpnode.execute("kubectl delete -f e2sar-headless-fabric.yaml")
    print(f"stderr: {stderr}")
except Exception as e:
    print(f"Exception: {e}")
    
    

namespace "e2sar-perf" deleted
persistentvolume "logs-pv" deleted
persistentvolumeclaim "logs-pvc" deleted
service "e2sar-receiver-svc" deleted
networkpolicy.networking.k8s.io "e2sar-network-policy" deleted
job.batch "e2sar-receiver" deleted
job.batch "e2sar-sender" deleted
job.batch "e2sar-log-viewer" deleted
stderr: 


## Issue: communication between cp and kubelet port on workers.
The control plane node cannot communicate with kubelet on port 10250 on worker nodes, despite being able to ping the worker node IP addresses. This prevents certain kubectl commands from functioning properly, particularly those that require direct kubelet communication, such as: "kubectl exec -it <pod-running-on-worker-node>".


# Delete Slice

Please delete your slicd when you are done with your experiment.

In [48]:
try:
    slice = fablib.get_slice(slice_name)
    slice.delete()
except Exception as e:
    print(f"Fail: {e}")